<a href="https://colab.research.google.com/github/angelaaaateng/ML_COVID_PREDICTION/blob/master/Exploration_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries + Load Data

In [37]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# amber additions
!pwd

/Users/angelateng/NYU_GoogleDrive/GitHub/ML_COVID_PREDICTION


### DSP Data (Key = PHCaseNo)

In [3]:
data = pd.read_csv('openrefine_PHCASES_COVID_04162020_PHnum.csv')
data.head()

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History 1,Travel History 2,Travel History 3,Travel History 4,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,NaN,NaN,NaN,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,NaN,NaN,NaN,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,NaN,NaN,NaN,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [4]:
data.columns

Index(['Case No.', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Travel History 1', 'Travel History 2',
       'Travel History 3', 'Travel History 4', 'Epi_Link',
       'Date of Onset of Symptoms', 'Date of Admission',
       'Date of Lab Confirmation', 'Date of Announcement to the Public',
       'Admission / Consultation', 'Symptoms', 'Other disease', 'Status',
       'Health Status', 'Date of Final Status (recovered/expired)',
       'Final Diagnosis', 'Age Group', 'Location', 'Location 1', 'Location 2',
       'Latitude', 'Longitude', 'Residence Lat', 'Residence Long'],
      dtype='object')

In [5]:
data = data.rename(columns={'Case No.': 'phcaseno'})

In [6]:
# data = data.rename(columns={'Location 1': 'case_location'})

In [7]:
data.dtypes

phcaseno                                     object
Sex                                          object
Age                                           int64
Nationality                                  object
Residence in the Philippines                 object
History of Travel / Exposure                 object
Travel History 1                             object
Travel History 2                             object
Travel History 3                             object
Travel History 4                             object
Epi_Link                                     object
Date of Onset of Symptoms                    object
Date of Admission                            object
Date of Lab Confirmation                     object
Date of Announcement to the Public           object
Admission / Consultation                     object
Symptoms                                     object
Other disease                                object
Status                                       object
Health Statu

In [8]:
data.shape

(3246, 30)

## Diagnosis Descriptions and Variables 

In [9]:
data.groupby('Health Status').size()

Health Status
asymptomatic        38
critical             3
died               108
for validation    2983
mild                57
recovered           51
severe               6
dtype: int64

## Feature Engineering: Symptoms

In [10]:
data.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History 1,Travel History 2,Travel History 3,Travel History 4,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,NaN,NaN,NaN,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),china,NaN,NaN,NaN,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,NaN,NaN,NaN,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,NaN,NaN,NaN,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [12]:
data_clean = data.drop(['Travel History 1',
 'Travel History 2',
 'Travel History 3',
 'Travel History 4'], axis = 1) 
data_clean.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Location 1,Location 2,Latitude,Longitude,Residence Lat,Residence Long
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sun Mar 15 00:00:00 EDT 2020,NaN,31 - 40,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sat Feb 01 00:00:00 EST 2020,severe pneumonia,41 - 50,Manila City,Manila City,NaN,14.613480,120.980950,14.598727,120.9819909
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,Sun Mar 15 00:00:00 EDT 2020,NaN,51 - 60,Manila City,Manila City,NaN,14.615867,120.980991,14.598727,120.9819909
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,Thu Mar 19 00:00:00 EDT 2020,"asymptomatic, negative test result",41 - 50,Taguig City,Taguig City,NaN,14.606904,121.020392,14.517274,121.0496133
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,Thu Mar 12 00:00:00 EDT 2020,"acute respiratory distress syndrome, severe pn...",61 - 70,"Cainta, Rizal",Cainta,Rizal,14.409635,121.037425,14.586506,121.1138255


In [48]:
# data_clean['phcaseno'].values.tolist()

In [15]:
data_clean.to_csv('/Users/angelateng/NYU_GoogleDrive/GitHub/ML_COVID_PREDICTION/data_clean.csv', index= False, header=True)

### One Hot Encoding Symptoms

In [21]:
data_clean.columns

Index(['phcaseno', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Epi_Link', 'Date of Onset of Symptoms',
       'Date of Admission', 'Date of Lab Confirmation',
       'Date of Announcement to the Public', 'Admission / Consultation',
       'Symptoms', 'Other disease', 'Status', 'Health Status',
       'Date of Final Status (recovered/expired)', 'Final Diagnosis',
       'Age Group', 'Location', 'Location 1', 'Location 2', 'Latitude',
       'Longitude', 'Residence Lat', 'Residence Long'],
      dtype='object')

In [23]:
symptoms = data_clean.set_index('phcaseno').Symptoms.str.split(',', expand=True).stack()

In [24]:
symptoms 

phcaseno   
PH00002   0                       fever
          1                       cough
          2                 sore throat
PH00004   0                       fever
PH00005   0                       cough
PH00006   0                       cough
          1     difficulty in breathing
PH00008   0                       fever
          1                 sore throat
PH00010   0                       fever
          1                       cough
PH00011   0                       cough
          1                 comorbidity
PH00012   0                       fever
          1                       cough
PH00013   0                       fever
PH00014   0                       fever
          1                       cough
PH00015   0                       cough
PH00016   0                       cough
          1                 sore throat
          2                       colds
PH00017   0                       cough
          1                       colds
PH00018   0                 

In [31]:
symptoms= pd.get_dummies(symptoms, prefix='symp').groupby(level=0).sum()
symptoms

,symp_ body malaise,symp_ colds,symp_ comorbidity,symp_ cough,symp_ difficulty in breathing,symp_ dizziness,symp_ dysphagia,symp_ headache,symp_ shortness of breath,symp_ sore throat,symp_asymptomatic,symp_cough,symp_difficulty of breathing,symp_dyspnea,symp_fever,symp_sore throat
phcaseno,,,,,,,,,,,,,,,,
PH00002,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
PH00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
PH00005,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
PH00006,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
PH00008,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
PH00010,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
PH00011,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
PH00012,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
PH00013,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [32]:
# join it back to the original 

clean_symptoms = data_clean.merge(symptoms, on=['phcaseno'], how='left', indicator=True)

In [33]:
clean_symptoms.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,symp_ headache,symp_ shortness of breath,symp_ sore throat,symp_asymptomatic,symp_cough,symp_difficulty of breathing,symp_dyspnea,symp_fever,symp_sore throat,_merge
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,both
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,both
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,PH03242,F,65,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3242,PH03243,F,24,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3243,PH03244,M,48,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3244,PH03245,M,67,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [34]:
clean_symptoms.columns

Index(['phcaseno', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Epi_Link', 'Date of Onset of Symptoms',
       'Date of Admission', 'Date of Lab Confirmation',
       'Date of Announcement to the Public', 'Admission / Consultation',
       'Symptoms', 'Other disease', 'Status', 'Health Status',
       'Date of Final Status (recovered/expired)', 'Final Diagnosis',
       'Age Group', 'Location', 'Location 1', 'Location 2', 'Latitude',
       'Longitude', 'Residence Lat', 'Residence Long', 'symp_ body malaise',
       'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat', '_merge'],
      dtype='object')

In [35]:
cols = ['symp_ body malaise',
       'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat']

In [43]:
clean_symptoms[cols] = clean_symptoms[cols].replace({np.nan:0.0})

**Table with One Hot Encoded Symptoms** 

In [45]:
clean_symptoms.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,symp_ headache,symp_ shortness of breath,symp_ sore throat,symp_asymptomatic,symp_cough,symp_difficulty of breathing,symp_dyspnea,symp_fever,symp_sore throat,_merge
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,both
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,both
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,PH03242,F,65,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3242,PH03243,F,24,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3243,PH03244,M,48,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3244,PH03245,M,67,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only


In [50]:
clean_symptoms = clean_symptoms.drop(['_merge'], axis =1)
clean_symptoms.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,symp_ dysphagia,symp_ headache,symp_ shortness of breath,symp_ sore throat,symp_asymptomatic,symp_cough,symp_difficulty of breathing,symp_dyspnea,symp_fever,symp_sore throat
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [51]:
clean_symptoms.columns

Index(['phcaseno', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Epi_Link', 'Date of Onset of Symptoms',
       'Date of Admission', 'Date of Lab Confirmation',
       'Date of Announcement to the Public', 'Admission / Consultation',
       'Symptoms', 'Other disease', 'Status', 'Health Status',
       'Date of Final Status (recovered/expired)', 'Final Diagnosis',
       'Age Group', 'Location', 'Location 1', 'Location 2', 'Latitude',
       'Longitude', 'Residence Lat', 'Residence Long', 'symp_ body malaise',
       'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat'],
      dtype='object')

In [ ]:
clean_symptoms['Other disease'].values.tolist()
clean_symptoms = clean_symptoms.rename(columns={'Other disease':'other_disease'}, errors="raise")


In [58]:
clean_symptoms.columns

Index(['phcaseno', 'Sex', 'Age', 'Nationality', 'Residence in the Philippines',
       'History of Travel / Exposure', 'Epi_Link', 'Date of Onset of Symptoms',
       'Date of Admission', 'Date of Lab Confirmation',
       'Date of Announcement to the Public', 'Admission / Consultation',
       'Symptoms', 'other_disease', 'Status', 'Health Status',
       'Date of Final Status (recovered/expired)', 'Final Diagnosis',
       'Age Group', 'Location', 'Location 1', 'Location 2', 'Latitude',
       'Longitude', 'Residence Lat', 'Residence Long', 'symp_ body malaise',
       'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat'],
      dtype='object')

In [59]:
disease = clean_symptoms.set_index('phcaseno').other_disease.str.split(',', expand=True).stack()
disease

phcaseno   
PH00005   0         acute kidney injury
          1                hypertension
          2                    diabetes
PH00006   0                    diabetes
PH00009   0                hypertension
                         ...           
PH01723   1     coronary artery disease
PH01800   0                    diabetes
PH01811   0                hypertension
PH02045   0                hypertension
PH02415   0                        none
Length: 213, dtype: object

In [60]:
disease_all= pd.get_dummies(disease, prefix='dis').groupby(level=0).sum()
disease_all

,dis_ diabetes,dis_ renal disease,dis_ acute kidney injury,dis_ asthma,dis_ atherosclerosis,dis_ atherosclerotic cardiovascular disease,dis_ benign prostatic hyperplasia,dis_ benign prostatic hypertrophy,dis_ bronchial asthma,dis_ cardiac disease,...,dis_hypertensive cardiovascular disease,dis_hyperthyroidism,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified)
phcaseno,,,,,,,,,,,,,,,,,,,,,
PH00005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH00006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH00009,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH00010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH00011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PH01723,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH01800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PH01811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
clean_disease = clean_symptoms.merge(disease_all, on=['phcaseno'], how='left', indicator=True)

In [64]:
list(clean_disease.columns)

['phcaseno',
 'Sex',
 'Age',
 'Nationality',
 'Residence in the Philippines',
 'History of Travel / Exposure',
 'Epi_Link',
 'Date of Onset of Symptoms',
 'Date of Admission',
 'Date of Lab Confirmation',
 'Date of Announcement to the Public',
 'Admission / Consultation',
 'Symptoms',
 'other_disease',
 'Status',
 'Health Status',
 'Date of Final Status (recovered/expired)',
 'Final Diagnosis',
 'Age Group',
 'Location',
 'Location 1',
 'Location 2',
 'Latitude',
 'Longitude',
 'Residence Lat',
 'Residence Long',
 'symp_ body malaise',
 'symp_ colds',
 'symp_ comorbidity',
 'symp_ cough',
 'symp_ difficulty in breathing',
 'symp_ dizziness',
 'symp_ dysphagia',
 'symp_ headache',
 'symp_ shortness of breath',
 'symp_ sore throat',
 'symp_asymptomatic',
 'symp_cough',
 'symp_difficulty of breathing',
 'symp_dyspnea',
 'symp_fever',
 'symp_sore throat',
 'dis_  diabetes',
 'dis_  renal disease',
 'dis_ acute kidney injury',
 'dis_ asthma',
 'dis_ atherosclerosis',
 'dis_ atherosclerotic 

In [66]:
cols = ['dis_  diabetes',
 'dis_  renal disease',
 'dis_ acute kidney injury',
 'dis_ asthma',
 'dis_ atherosclerosis',
 'dis_ atherosclerotic cardiovascular disease',
 'dis_ benign prostatic hyperplasia',
 'dis_ benign prostatic hypertrophy',
 'dis_ bronchial asthma',
 'dis_ cardiac disease',
 'dis_ cardiac dysrhythmia',
 'dis_ cardiomyopathy',
 'dis_ cardiovascular disease',
 'dis_ cerebrovascular accident infarc',
 'dis_ cerebrovascular disease infarction',
 'dis_ chronic kidney disease',
 'dis_ chronic obstructive pulmonary disease',
 'dis_ colon cancer',
 'dis_ coronary artery disease',
 'dis_ coronary heart disease',
 'dis_ dengue',
 'dis_ diabetes',
 'dis_ diabetes mellitus',
 'dis_ diabetic ketoacidosis',
 'dis_ diabetic nephropathy',
 'dis_ diabetis mellitus',
 'dis_ dyslipidemia',
 'dis_ endocrine condition',
 'dis_ hypertension',
 'dis_ hypertensive cardiovascular disease',
 'dis_ hypertensive nephrosclerosis',
 'dis_ hyperthyroidism',
 'dis_ hypothyroidism',
 'dis_ impaired fasting glucose',
 'dis_ ischemic heart disease',
 'dis_ kidney disease',
 'dis_ peripheral arterial occlusive disease',
 'dis_ renal disease',
 'dis_ tongue cancer',
 'dis_ valvular heart disease',
 'dis_acute kidney injury',
 'dis_acute kidney injury secondary to pre-renal azotemia',
 'dis_acute renal failure',
 'dis_asthma',
 'dis_asymptomatic',
 'dis_breast cancer',
 'dis_bronchial asthma',
 'dis_cardiac disease',
 'dis_chronic kidney disease',
 'dis_chronic kidney disease secondary to hypertension nephrosclerosis',
 'dis_chronic obstructive pulmonary disease',
 'dis_community-acquired pneumonia',
 'dis_diabetes',
 'dis_hypertension',
 'dis_hypertensive',
 'dis_hypertensive cardiovascular disease',
 'dis_hyperthyroidism',
 'dis_myocarditis',
 'dis_none',
 'dis_permanent atrial fibrillation',
 'dis_prostatic cancer',
 'dis_remnal disease',
 'dis_status post renal transplant',
 'dis_systemic lupus erythematosus',
 'dis_with comorbid (not specified)']

In [67]:
clean_disease[cols] = clean_disease[cols].replace({np.nan:0.0})

In [68]:
clean_disease

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,dis_hyperthyroidism,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified),_merge
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,PH03242,F,65,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3242,PH03243,F,24,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3243,PH03244,M,48,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only
3244,PH03245,M,67,filipino,For validation,0,NaN,NaN,NaN,Fri Apr 03 00:00:00 EDT 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,left_only


**The `symptoms` and `other_disease` columns have no been feature engineered.**

In [70]:
clean_disease = clean_disease.drop(['_merge'], axis = 1) 
clean_disease.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Epi_Link,Date of Onset of Symptoms,Date of Admission,Date of Lab Confirmation,...,dis_hypertensive cardiovascular disease,dis_hyperthyroidism,dis_myocarditis,dis_none,dis_permanent atrial fibrillation,dis_prostatic cancer,dis_remnal disease,dis_status post renal transplant,dis_systemic lupus erythematosus,dis_with comorbid (not specified)
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,Wife of PH2,Tue Jan 21 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,Husband of PH1,Sat Jan 18 00:00:00 EST 2020,Sat Jan 25 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PH00003,F,60,chinese,None,Yes (China),Traveled to Cebu and Bohol. Was already in Chi...,Tue Jan 21 00:00:00 EST 2020,Wed Jan 22 00:00:00 EST 2020,Thu Jan 30 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),NaN,Tue Mar 03 00:00:00 EST 2020,Fri Mar 06 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,Husband of PH6,Tue Feb 25 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,Thu Mar 05 00:00:00 EST 2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


@TODO: Add a new column based on the number of symptoms and the number of other diseases. 

In [84]:
clean_disease.shape

(3246, 107)

## Feature Engineering Location

In [104]:
# %load_ext au totime
import pandas as pd

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
import time

import plotly_express as px

In [120]:
geolocator = Nominatim()
# geolocator = Nominatim(domain='localhost:8888/nominatim', scheme='http')
# geolocator=Nominatim(domain='localhost:8080', scheme='http')

In [121]:
# clean_disease['geo_address'] = clean_disease.apply(lambda row: Nominatim.reverse_geocode(row['Latitude'],row['Longitude']),axis=1)

In [123]:
for index, row in clean_disease.iterrows():
    location = geolocator.reverse("{}, {}".format(row['Latitude'], row['Longitude']) , timeout = 100)
    time.sleep(2)
    geo_string = location.address.split(",")
    print(index/len(clean_disease))
    

GeocoderServiceError: [Errno 61] Connection refused

In [107]:
len(clean_disease)

3246

In [125]:
geo_string 

['Main Building',
 ' Wilson',
 ' Greenhills',
 ' Addition Hills',
 ' 2nd District',
 ' San Juan',
 ' Metro Manila',
 ' 1502',
 ' Philippines']